In [1]:
import lib.facebook_scraper as fs
import lib.utils as utils
import pandas as pd
import numpy as np
import os

In [2]:
post_ids_df = pd.read_csv('data/vebay69/clean/post_ids.csv')
post_ids_df.head()

,post_id
0,685638947004646
1,685577330344141
2,685111467057394
3,684686633766544
4,684603707108170


In [3]:
index_post_id_file_path = os.path.join('data','vebay69','url','i_post_id_share.txt')

In [4]:
def extract_sharer(sharers, post_id):
    cnt_sharer = 0
    global sharer_list
    for sharer in sharers:
        sharer['post_id'] = post_id
        sharer_list.append(sharer)
        cnt_sharer += 1
        # time.sleep(np.random.uniform(0, 1))
    return cnt_sharer

## Rerun with change cookie

In [5]:
fs.set_cookies('./cookies/cookie_1.txt')
# fs.enable_logging()

## Rerun

In [6]:
NUMBER_POST = 100
post_list = []
sharer_list = []
total_sharer_real = 0

In [7]:
index_post_id = utils.read_index_file(file_path=index_post_id_file_path)
print(index_post_id)
posts_ids = utils.get_post_id_list(df=post_ids_df, start=index_post_id, length=NUMBER_POST)
posts_ids

903


[5271694419617373,
 5271402066313275,
 5271002853019863,
 5269137123206436,
 5266711463449002,
 5263709253749223,
 5262746583845490]

In [8]:
try:
    for post in fs.get_posts(
        post_urls=posts_ids,
        options={
            "allow_extra_requests": True,
            "sharers": True,
            "sharers": "generator",
        },
        timeout=120,
    ):
        print(post)
        num_sharer = extract_sharer(post['sharers'], post['original_request_url'])
        post_list.append(post)
        print(f'Extracted {num_sharer}/{post["shares"]} sharers')
        total_sharer_real += post['shares']
        utils.sleep(np.random.randint(5, 10))
except fs.exceptions.TemporarilyBanned:
    print("TEMPORARILY BANNED")
except fs.exceptions.AccountDisabled:
    print("ACCOUNT DISABLED")
except Exception as e:
    print(e)

/workspaces/analyze-facebook-page/lib/facebook_scraper/facebook_scraper.py:912: UserWarning: Facebook says 'Unsupported Browser'
  warnings.warn(f"Facebook says 'Unsupported Browser'")


{'original_request_url': 5271694419617373, 'post_url': 'https://facebook.com/story.php?story_fbid=pfbid02p1N154im5UZS7jDi6kKZs9n1c3rHpoznnLV7wXD9jKSyif92V6Z3Dewfz3LVN318l&id=772954189491441', 'post_id': 'pfbid02p1N154im5UZS7jDi6kKZs9n1c3rHpoznnLV7wXD9jKSyif92V6Z3Dewfz3LVN318l', 'text': 'Tôi đen quá\n#Lìquầnxòi', 'post_text': 'Tôi đen quá\n#Lìquầnxòi', 'shared_text': '', 'original_text': None, 'time': datetime.datetime(2022, 5, 9, 10, 32, 5), 'timestamp': 1652092325, 'image': 'https://scontent-sin6-2.xx.fbcdn.net/v/t39.30808-6/280499748_5271694172950731_814888226898842408_n.png?_nc_cat=105&ccb=1-7&_nc_sid=5f2048&efg=eyJpIjoidCJ9&_nc_eui2=AeFDfBOuH429NU7p7MxuA27C3jN0Ij3Ef9XeM3QiPcR_1bqhA-x6ldL1iDCXvW9hBuLI3pWxEoKf6FfAxAihtvbN&_nc_ohc=y3Oin2ZSxwAAX9lpCCm&_nc_ht=scontent-sin6-2.xx&cb_e2o_trans=q&oh=00_AfDMPrh-8po8t0KLT0YvvZ32QxZRhDPWwLzWZcbBhNDM2g&oe=65688EA4', 'image_lowquality': 'https://scontent-sin6-2.xx.fbcdn.net/v/t39.30808-6/280499748_5271694172950731_814888226898842408_n.png?stp=ds

In [9]:
print(NUMBER_POST)
print(f"Số posts đã crawl: {len(post_list)}")
print(f"Số sharers đã crawl: {len(sharer_list)}/{total_sharer_real}")

100
Số posts đã crawl: 7
Số sharers đã crawl: 85/239


In [10]:
if sharer_list:
    utils.save_data(
        data_list=sharer_list,
        folder_path=os.path.join('data','vebay69','raw'),
        type="sharers"
    )
utils.write_index_file(
    file_path=index_post_id_file_path,
    index=index_post_id + len(post_list)
)

Save sharers data: ./data/vebay69/raw/sharers_2023-11-26_12-24-41.csv
Save index 910 to resume: ./data/vebay69/url/i_post_id_share.txt
